In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {world: idx for idx, world in enumerate(world_set)}

In [4]:
print(vocab)

{' ': 0, 'a': 1, ',': 2, 'm': 3, 'B': 4, 'c': 5, '.': 6, 'o': 7, 's': 8, 'h': 9, 'e': 10, 'f': 11, 'n': 12, 'r': 13, 'b': 14, 'p': 15, 'l': 16, "'": 17, 'k': 18, 'y': 19, 'i': 20, 'u': 21, 'd': 22, 't': 23, 'g': 24, 'w': 25}


In [5]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [6]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [7]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [8]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[4, 13, 20, 5, 18, 0, 25, 1, 16, 16]
[13, 20, 5, 18, 0, 25, 1, 16, 16, 8]


In [9]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
# np.eye 는 대각선이 1인 행렬 생성하는 함수
x_data_np = np.array(x_data)
x_one_hot = np.eye(len(vocab))[x_data_np]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) # 원핫인코딩된 데이터, 각각의 문자에 대한 확률
Y = torch.LongTensor(y_data) #각 문자에 대한 정수 인덱스, 클래스 레이블

In [10]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [11]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [12]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([13, 20,  5, 18,  0, 25,  1, 16, 16,  8])


In [13]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _ = self.rnn(x)
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [14]:
net = Net(vocab_size, hidden_size, 2)

In [15]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [16]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [17]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, len(vocab)), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

uffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
aaa aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
a                                                                                                                                                                                                    
a                                                                                                                                                                                                    
                                                                                                                                                                                                     
          

                                                     e     e    e                      e                           e                e    e                                                           
                                                     e     e    e                      e                           e                e    e                            e                         e    
                 e                                  he    he   he                     he                        h he       e       he   he                   e        e                        he    
             e   e                                  he    he   he               e     he                        h he       e       he   he               e   e        e                        he    
     t       e  he          h                 et    he   the   he       t       e     he    h      t     h      h he     the   h   he   he       t       e  he    h   et         h            the    
  t  t t  

 eck talls are the e tht a meec s are tht aets tht thetk thet the toisk talls are  ththe e th tee  ts  is  tst  ethe  t  thet tat thet the toisk talls are the e th t et as  et aatls te ta l thetks 
 eck talls are the e tht arkeecos are aot aats tot theck thet the trisk talls are  ththe e th tee  as  is  tat let er tsothet tat thetothe trisk talls are the e th t et as  et aatls ae ta l thecks 
 eck talls are the e th  a leec   arl tht aets tot theck thet the trick talls are  ththe e th tee  as  is  tst  ethe  tr thet tal thet the trick talls are the e th t et as  et aalls te ta l theck  
 eck talls are the e tht arleec s ara aht aats tot theck thet the trick talls are  ththe e th teen as  is  tat  ether tr thet tat thet the trick talls are the e th t et as  et aatls ae tall thecks 
 eck talls are the e tht arleec s are aht aats tet theck thet the trick talls are  t the e th teer as  is  tat  ether tr thec tat thet the trick talls are the e th t et as  et aatls te tall thecks 
 eck talls

In [18]:
predict_str

' eck talls are the e thr arleec n are aot auts  et theck thet the trick talls are ta the e th teen as  is  tat  et er ar thet tat thet the trick talls are the e th t et as  et aatls te talt thecks '

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요? A. 꽤나요,,, 오

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요!

영어가 아닌 한국어로 시도해보는 것도 좋겠죠?

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [19]:
sentence = ("나의 여름이 모든 색을 잃고 흑백이 되어도 좋습니다 "
            "내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 "
            "그러니 그대는 나의 여름이 되세요.")

In [20]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [21]:
import konlpy
from konlpy.tag import Okt, Mecab

In [22]:
tokenizer=Okt()
print(tokenizer.morphs(sentence))

['나', '의', '여름', '이', '모든', '색', '을', '잃고', '흑백', '이', '되어도', '좋습니다', '내', '가', '세상', '의', '꽃', '들', '과', '들풀', ',', '숲', '의', '색', '을', '모두', '훔쳐', '올', '테', '니', '전부', '그대', '의', '것', '하십시오', '그러니', '그대', '는', '나', '의', '여름', '이', '되세요', '.']


In [23]:
#world_set = list(set(sentence))
world_set = list(set(sentence))
print(world_set)
vocab = {world: idx for idx, world in enumerate(world_set)}

['가', '쳐', '시', '나', ' ', '두', '하', '는', '훔', '고', '다', ',', '을', '.', '올', '여', '십', '의', '되', '요', '든', '꽃', '대', '내', '흑', '이', '잃', '전', '그', '오', '니', '숲', '상', '과', '러', '테', '색', '습', '풀', '부', '들', '것', '세', '좋', '백', '도', '름', '모', '어']


In [24]:
print(vocab)

{'가': 0, '쳐': 1, '시': 2, '나': 3, ' ': 4, '두': 5, '하': 6, '는': 7, '훔': 8, '고': 9, '다': 10, ',': 11, '을': 12, '.': 13, '올': 14, '여': 15, '십': 16, '의': 17, '되': 18, '요': 19, '든': 20, '꽃': 21, '대': 22, '내': 23, '흑': 24, '이': 25, '잃': 26, '전': 27, '그': 28, '오': 29, '니': 30, '숲': 31, '상': 32, '과': 33, '러': 34, '테': 35, '색': 36, '습': 37, '풀': 38, '부': 39, '들': 40, '것': 41, '세': 42, '좋': 43, '백': 44, '도': 45, '름': 46, '모': 47, '어': 48}


In [25]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 49


In [26]:
hidden_size = vocab_size
sequence_length = 10
learning_rate = 0.01

In [27]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 나의 여름이 모든  -> 의 여름이 모든 색
1 의 여름이 모든 색 ->  여름이 모든 색을
2  여름이 모든 색을 -> 여름이 모든 색을 
3 여름이 모든 색을  -> 름이 모든 색을 잃
4 름이 모든 색을 잃 -> 이 모든 색을 잃고
5 이 모든 색을 잃고 ->  모든 색을 잃고 
6  모든 색을 잃고  -> 모든 색을 잃고 흑
7 모든 색을 잃고 흑 -> 든 색을 잃고 흑백
8 든 색을 잃고 흑백 ->  색을 잃고 흑백이
9  색을 잃고 흑백이 -> 색을 잃고 흑백이 
10 색을 잃고 흑백이  -> 을 잃고 흑백이 되
11 을 잃고 흑백이 되 ->  잃고 흑백이 되어
12  잃고 흑백이 되어 -> 잃고 흑백이 되어도
13 잃고 흑백이 되어도 -> 고 흑백이 되어도 
14 고 흑백이 되어도  ->  흑백이 되어도 좋
15  흑백이 되어도 좋 -> 흑백이 되어도 좋습
16 흑백이 되어도 좋습 -> 백이 되어도 좋습니
17 백이 되어도 좋습니 -> 이 되어도 좋습니다
18 이 되어도 좋습니다 ->  되어도 좋습니다 
19  되어도 좋습니다  -> 되어도 좋습니다 내
20 되어도 좋습니다 내 -> 어도 좋습니다 내가
21 어도 좋습니다 내가 -> 도 좋습니다 내가 
22 도 좋습니다 내가  ->  좋습니다 내가 세
23  좋습니다 내가 세 -> 좋습니다 내가 세상
24 좋습니다 내가 세상 -> 습니다 내가 세상의
25 습니다 내가 세상의 -> 니다 내가 세상의 
26 니다 내가 세상의  -> 다 내가 세상의 꽃
27 다 내가 세상의 꽃 ->  내가 세상의 꽃들
28  내가 세상의 꽃들 -> 내가 세상의 꽃들과
29 내가 세상의 꽃들과 -> 가 세상의 꽃들과 
30 가 세상의 꽃들과  ->  세상의 꽃들과 들
31  세상의 꽃들과 들 -> 세상의 꽃들과 들풀
32 세상의 꽃들과 들풀 -> 상의 꽃들과 들풀,
33 상의 꽃들과 들풀, -> 의 꽃들과 들풀, 
34 의 꽃들과 들풀,  ->  꽃들과 들풀, 숲
35  꽃들과 들풀, 숲 -> 꽃들과 들풀, 숲의
36

In [28]:
print(x_data[0])
print(y_data[0])

[3, 17, 4, 15, 46, 25, 4, 47, 20, 4]
[17, 4, 15, 46, 25, 4, 47, 20, 4, 36]


In [29]:
x_data_np = np.array(x_data)
x_one_hot = np.eye(len(vocab))[x_data_np]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [30]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([83, 10, 49])
레이블의 크기 : torch.Size([83, 10])


In [31]:
print(X[0])

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.

In [32]:
print(Y[0])

tensor([17,  4, 15, 46, 25,  4, 47, 20,  4, 36])


In [33]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _ = self.rnn(x)
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [34]:
net = Net(vocab_size, hidden_size, 2)

In [35]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [36]:
outputs = net(X)
print(outputs.shape)

torch.Size([83, 10, 49])


In [37]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, len(vocab)), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([world_set[t] for t in result])
        else:
            predict_str += world_set[result[-1]]

    print(predict_str)

들들들들들들들들들들들들들들들들들십들들들들들들들들들되들들들들들들들들들들들들들들들들들들들들들들들들들들들들들십들들들들들들들들들들들들들들들들들들들들들들들들들들들들들들들들들들
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                      

의 여름이 모든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.
의 여름이 되든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.
의 여름이 되든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.
의 여름이 모든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.
의 여름이 되든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.
의 여름이 되든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.
의 여름이 모든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.
의 여름이 되든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.
의 여름이 모든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.


In [38]:
predict_str

'의 여름이 모든 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여름이 되세요.'

시도 2

In [39]:
sentence = ("나의 여름이 모든 색을 잃고 흑백이 되어도 좋습니다 "
            "내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐올 테니 전부 그대의 것 하십시오 "
            "그러니 그대는 나의 여름이 되세요.")

In [40]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [41]:
import konlpy
from konlpy.tag import Okt, Mecab

In [42]:
tokenizer=Okt()
print(tokenizer.morphs(sentence))

['나', '의', '여름', '이', '모든', '색', '을', '잃고', '흑백', '이', '되어도', '좋습니다', '내', '가', '세상', '의', '꽃', '들', '과', '들풀', ',', '숲', '의', '색', '을', '모두', '훔쳐', '올', '테', '니', '전부', '그대', '의', '것', '하십시오', '그러니', '그대', '는', '나', '의', '여름', '이', '되세요', '.']


In [43]:
#world_set = list(set(sentence))
world_set = list(set(sentence))
print(world_set)
vocab = {world: idx for idx, world in enumerate(world_set)}

['가', '쳐', '시', '나', ' ', '두', '하', '는', '훔', '고', '다', ',', '을', '.', '올', '여', '십', '의', '되', '요', '든', '꽃', '대', '내', '흑', '이', '잃', '전', '그', '오', '니', '숲', '상', '과', '러', '테', '색', '습', '풀', '부', '들', '것', '세', '좋', '백', '도', '름', '모', '어']


In [44]:
print(vocab)

{'가': 0, '쳐': 1, '시': 2, '나': 3, ' ': 4, '두': 5, '하': 6, '는': 7, '훔': 8, '고': 9, '다': 10, ',': 11, '을': 12, '.': 13, '올': 14, '여': 15, '십': 16, '의': 17, '되': 18, '요': 19, '든': 20, '꽃': 21, '대': 22, '내': 23, '흑': 24, '이': 25, '잃': 26, '전': 27, '그': 28, '오': 29, '니': 30, '숲': 31, '상': 32, '과': 33, '러': 34, '테': 35, '색': 36, '습': 37, '풀': 38, '부': 39, '들': 40, '것': 41, '세': 42, '좋': 43, '백': 44, '도': 45, '름': 46, '모': 47, '어': 48}


In [45]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 49


In [46]:
hidden_size = vocab_size
sequence_length = 10
learning_rate = 0.01

In [47]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 나의 여름이 모든  -> 의 여름이 모든 색
1 의 여름이 모든 색 ->  여름이 모든 색을
2  여름이 모든 색을 -> 여름이 모든 색을 
3 여름이 모든 색을  -> 름이 모든 색을 잃
4 름이 모든 색을 잃 -> 이 모든 색을 잃고
5 이 모든 색을 잃고 ->  모든 색을 잃고 
6  모든 색을 잃고  -> 모든 색을 잃고 흑
7 모든 색을 잃고 흑 -> 든 색을 잃고 흑백
8 든 색을 잃고 흑백 ->  색을 잃고 흑백이
9  색을 잃고 흑백이 -> 색을 잃고 흑백이 
10 색을 잃고 흑백이  -> 을 잃고 흑백이 되
11 을 잃고 흑백이 되 ->  잃고 흑백이 되어
12  잃고 흑백이 되어 -> 잃고 흑백이 되어도
13 잃고 흑백이 되어도 -> 고 흑백이 되어도 
14 고 흑백이 되어도  ->  흑백이 되어도 좋
15  흑백이 되어도 좋 -> 흑백이 되어도 좋습
16 흑백이 되어도 좋습 -> 백이 되어도 좋습니
17 백이 되어도 좋습니 -> 이 되어도 좋습니다
18 이 되어도 좋습니다 ->  되어도 좋습니다 
19  되어도 좋습니다  -> 되어도 좋습니다 내
20 되어도 좋습니다 내 -> 어도 좋습니다 내가
21 어도 좋습니다 내가 -> 도 좋습니다 내가 
22 도 좋습니다 내가  ->  좋습니다 내가 세
23  좋습니다 내가 세 -> 좋습니다 내가 세상
24 좋습니다 내가 세상 -> 습니다 내가 세상의
25 습니다 내가 세상의 -> 니다 내가 세상의 
26 니다 내가 세상의  -> 다 내가 세상의 꽃
27 다 내가 세상의 꽃 ->  내가 세상의 꽃들
28  내가 세상의 꽃들 -> 내가 세상의 꽃들과
29 내가 세상의 꽃들과 -> 가 세상의 꽃들과 
30 가 세상의 꽃들과  ->  세상의 꽃들과 들
31  세상의 꽃들과 들 -> 세상의 꽃들과 들풀
32 세상의 꽃들과 들풀 -> 상의 꽃들과 들풀,
33 상의 꽃들과 들풀, -> 의 꽃들과 들풀, 
34 의 꽃들과 들풀,  ->  꽃들과 들풀, 숲
35  꽃들과 들풀, 숲 -> 꽃들과 들풀, 숲의
36

In [48]:
print(x_data[0])
print(y_data[0])

[3, 17, 4, 15, 46, 25, 4, 47, 20, 4]
[17, 4, 15, 46, 25, 4, 47, 20, 4, 36]


In [49]:
x_data_np = np.array(x_data)
x_one_hot = np.eye(len(vocab))[x_data_np]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [50]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([83, 10, 49])
레이블의 크기 : torch.Size([83, 10])


In [51]:
print(X[0])

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.

In [52]:
print(Y[0])

tensor([17,  4, 15, 46, 25,  4, 47, 20,  4, 36])


In [53]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, x):
    x, _ = self.rnn(x)
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [54]:
net = Net(vocab_size, hidden_size, 2)

In [55]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adagrad(net.parameters(), learning_rate)

In [56]:
outputs = net(X)
print(outputs.shape)

torch.Size([83, 10, 49])


In [57]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, len(vocab)), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([world_set[t] for t in result])
        else:
            predict_str += world_set[result[-1]]

    print(predict_str)

모오오오고오오오고오오고백오고고백고오오고오오오고고오고백오고고고오오오오고백고고백백백오고오오고오오고오백오백오오고고백고백백고오오오고오고고오오백고오고고고오오백고오오고오오고고오
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                      

  그대이 되어 색을 모고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러의 그대는 나의 여대이 되어, 
  그대이 되어 색을 잃고 흑백이 되어도 그습니다 내가 그상의 색들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러의 그대는 나의 그대이 되어, 
  그대이 되어 색을 모고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러의 그대는 나의 여대이 되어, 
  그대이 되어 색을 잃고 흑백이 되어도 좋습니다 내가 그상의 색들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러의 그대는 나의 그대이 되어, 
  그대이 되어 색을 모고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러의 그대는 나의 여대이 되어, 
  그대이 되어 색을 잃고 흑백이 되어도 좋습니다 내가 그상의 색들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러의 그대는 나의 여대이 되어, 
  그대이 되어 색을 모고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러의 그대는 나의 여대이 되어, 
  그대이 되어 색을 잃고 흑백이 되어도 좋습니다 내가 그상의 색들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러의 그대는 나의 여대이 되어, 
  그대이 되어 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여대이 되어, 
  그대이 되어 색을 잃고 흑백이 되어도 좋습니다 내가 그상의 색들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여대이 되어, 
  그대이 되어 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십

In [58]:
predict_str

'  그대이 되어 색을 잃고 흑백이 되어도 좋습니다 내가 세상의 꽃들과 들풀, 숲의 색을 모두 훔쳐이 테니 전부 그대의 것 하십시오 그러니 그대는 나의 여대이 되어,십'